# Imports

In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.1 MB/s eta 0:00:00


In [ ]:
! pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.2 MB/s eta 0:00:00


In [ ]:
# ============ COLAB SETUP ============
# from google.colab import drive
# drive.mount('/content/drive')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# HuggingFace Transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from pathlib import Path
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Sklearn tools
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score,precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import optuna
import wandb

import os
import re
import html
import emoji
import warnings
warnings.filterwarnings('ignore')

# Login to WandB
wandb.login()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yardenshalom (yardenshalom-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# === Load data ===
#train_df = pd.read_csv('/content/drive/MyDrive/deep_learning/train_processed.csv', encoding='latin1')
#eval_df = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv', encoding='latin1')
#test_df = pd.read_csv('/content/drive/MyDrive/deep_learning/test_processed.csv', encoding='latin1')


# Navigate from model folder to data folder
current_dir = Path.cwd()  # models/bert-base-uncased/
models_dir = current_dir.parent  # models/
project_root = models_dir.parent  # project root
data_dir = models_dir / 'data'

# Load data
train_df = pd.read_csv(data_dir / 'train_processed.csv', encoding='latin1')
eval_df = pd.read_csv(data_dir / 'val_processed.csv', encoding='latin1')
test_df = pd.read_csv(data_dir / 'test_processed.csv', encoding='latin1')

print(f"Data loaded from: {data_dir}")
print(f"Train shape: {train_df.shape}")
print(f"Val shape: {eval_df.shape}")
print(f"Test shape: {test_df.shape}")


# Create label mappings
ordered_labels = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
label2id = {label: i for i, label in enumerate(ordered_labels)}
id2label = {i: label for label, i in label2id.items()}

train_df["label"] = train_df["Sentiment"].map(label2id)
eval_df["label"] = eval_df["Sentiment"].map(label2id)
test_df["label"] = test_df["Sentiment"].map(label2id)


In [ ]:
# Display
pd.set_option('display.max_colwidth', None)  # so full text is shown
sample = train_df[['ProcessedTweet','Sentiment','label']].sample(10, random_state=24)
display(sample)

,ProcessedTweet,Sentiment,label
17977,new survey by user finds that only 2% of #consumers think #brands should pause all #advertising; 49% want #ads to make them feel informed and 37% want ads to make them feel warm/happy #digitalmarketing #covid #consumerbehavior http,Extremely Positive,4
31793,some #costco stores say their prohibiting people from returning #toiletpaper. who returns toilet paper? probably the same people who think their rectums go into remission? #trending #covid user #jimmy #kimmel #for #real,Neutral,2
8808,"after increasing the prices of platform tickets, the indian railways withdrew most concessional ticket facilities in trains as a precautionary measure to contain the spread of #covid. http",Neutral,2
23027,"user money hdelacerda. oil prices caused a massive slow down and now the covid 19 has caused layoffs, i havent worked since the beginning of february just trying to make sure my wife and 2 daughters are taken care of.",Extremely Positive,4
6409,my wife just messaged me to say the supermarket shelves were empty. i know what's for dinner tonight #covid http,Negative,1
28137,"i nominate the global coronavirus responders for the 2020 time magazines person of the year: (including nurses, paramedics, doctors, hospital admin and cleaning staff, grocery store employees, etc) #covid",Neutral,2
22277,"wed like to take a moment to thank all of the grocery store personnel, cleaners, maintenance workers, truck drivers and warehouse workers for their perseverance and commitment. we are incredibly grateful to those who are making sacrifices to support the covid 19 fight. http",Extremely Positive,4
21206,this is too much! no fizzy san pelegrno water for my wine spritzer or houmous! what in gods name do you expect the middle classes to do! to die like dinosaurs! #covid #stay #home #24 #in #48 #stockpiling #uk #toiletpaper #covid,Extremely Negative,0
15208,? money cag money gis money k money ko consumer brands association ceo on the state of global supply chains amid covid 19 http,Neutral,2
28598,breaking: govt. has announced good news for shoppers who have been battling to buy essential products at inflated prices. retailers on the naughty list could face hefty fines or even prison sentences. #covid http,Negative,1


--------

In [ ]:
class CoronaTweetDataset(Dataset):
    """
    Dataset class for loading COVID-19 tweets for sentiment classification.
    Compatible with HuggingFace transformers and PyTorch.
    """
    def __init__(self, dataframe, tokenizer):
        self.texts = dataframe['ProcessedTweet'].tolist()  # Cleaned tweets
        self.labels = dataframe['label'].tolist()          # Encoded sentiment labels
        self.tokenizer = tokenizer                         # Tokenizer for the model

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize input text with padding and truncation
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=128,       # 128 is enough for tweets
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),             # Tensor of token IDs
            'attention_mask': encoding['attention_mask'].squeeze(),   # Tensor mask
            'labels': torch.tensor(label, dtype=torch.long)           # Ground truth label
        }


# Training

In [ ]:
def early_stop_check(patience, best_val_accuracy, best_val_accuracy_epoch, current_val_accuracy, current_val_accuracy_epoch):
    early_stop_flag = False
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy
        best_val_accuracy_epoch = current_val_accuracy_epoch
    else:
        if current_val_accuracy_epoch - best_val_accuracy_epoch > patience:
            early_stop_flag = True
    return best_val_accuracy, best_val_accuracy_epoch, early_stop_flag

In [ ]:
# ============ UPDATED TRAINING FUNCTION WITH GRADIENT CLIPPING ============
def train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs, patience, trial, gradient_clip_val=None):
    """
    Updated training function with gradient clipping and enhanced monitoring
    """
    best_val_accuracy = 0.0
    best_val_accuracy_epoch = 0
    early_stop_flag = False
    best_model_state = None

    for epoch in range(1, epochs + 1):
        # Training phase
        model.train()
        train_loss = 0.0
        total_train_samples = 0
        correct_train_predictions = 0

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            loss.backward()

            # NEW: Gradient Clipping
            if gradient_clip_val is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)

            optimizer.step()

            # Accumulate training metrics
            train_loss += loss.item() * input_ids.size(0)
            total_train_samples += input_ids.size(0)
            correct_train_predictions += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        # Validation phase
        model.eval()
        val_loss = 0.0
        total_val_samples = 0
        correct_val_predictions = 0
        all_val_labels = []
        all_val_preds = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                total_val_samples += input_ids.size(0)
                correct_val_predictions += (logits.argmax(dim=1) == labels).sum().item()

                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(logits.argmax(dim=1).cpu().numpy())

        # Calculate per-class metrics
        class_names = ['Extremely Negative', 'Extremely Positive', 'Negative','Neutral','Positive']
        precisions, recalls, f1s, _ = precision_recall_fscore_support( all_val_labels, all_val_preds, average=None, labels=[0, 1, 2, 3, 4], zero_division=0)

        # Log per-class metrics to wandb
        for i, class_name in enumerate(class_names):
            wandb.log({
                f"Class_{class_name}_Precision": precisions[i] if i < len(precisions) else 0.0,
                f"Class_{class_name}_Recall": recalls[i] if i < len(recalls) else 0.0,
                f"Class_{class_name}_F1": f1s[i] if i < len(f1s) else 0.0
            })

        # Calculate overall metrics
        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples
        val_precision = precision_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_recall = recall_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='macro', zero_division=0)

        # Early stopping check
        best_val_accuracy, best_val_accuracy_epoch, early_stop_flag = early_stop_check(
            patience, best_val_accuracy, best_val_accuracy_epoch, val_accuracy, epoch
        )

        # Save best model
        if val_accuracy == best_val_accuracy:
            best_model_state = model.state_dict()

        # Log metrics to wandb
        wandb.log({
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_accuracy,
            "Validation Precision": val_precision,
            "Validation Recall": val_recall,
            "Validation F1": val_f1
        })

        # Print progress
        print(f"Epoch {epoch:2d}/{epochs}: "
              f"Train Acc: {train_accuracy:.4f}, "
              f"Val Acc: {val_accuracy:.4f}, "
              f"Val F1: {val_f1:.4f}, "
              f"Gap: {(train_accuracy - val_accuracy):.4f}")

        if len(f1s) == 3:
            print(f"          Per-class F1: "
                  f"Neg: {f1s[0]:.3f}, "
                  f"Neu: {f1s[1]:.3f}, "
                  f"Pos: {f1s[2]:.3f}")

        if early_stop_flag:
            print(f"Early stopping at epoch {epoch}")
            break

    if best_model_state is not None:
        torch.save(best_model_state, f"best_model_trial_{trial.number}.pt")

    return best_val_accuracy


In [ ]:
# ============ CLASS WEIGHTS COMPUTATION ============
from sklearn.utils.class_weight import compute_class_weight
def compute_class_weights(train_labels):
    """
    Compute class weights for your imbalanced dataset
    """
    train_labels = np.array(train_labels)
    unique_labels = np.unique(train_labels)

    print(f"Computing class weights for {len(train_labels)} samples")
    print(f"Label distribution: {np.bincount(train_labels)}")

    class_weights = compute_class_weight(
        'balanced',
        classes=unique_labels,
        y=train_labels
    )

    print(f"Class weights computed:")
    for i, label in enumerate(unique_labels):
        label_name = ['Extremely Negative', 'Negative','Neutral','Positive','Extremely Positive'][label]
        print(f"{label_name} ({label}): {class_weights[i]:.3f}")

    return torch.tensor(class_weights, dtype=torch.float)


from transformers import AutoModel

class ConfigurableBertModel(nn.Module):
    def __init__(self, model_name: str, num_labels: int, dropout_rate: float = 0.1):
        super(ConfigurableBertModel, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return type('ModelOutput', (), {'logits': logits})()



### Hyperparameter Search — Broad Stage

We began the hyperparameter tuning process with a **broad search space**, allowing Optuna to explore a wide range of values for the main parameters, including learning rate, dropout, weight decay, warmup ratio, label smoothing, batch size, and scheduler type.
The purpose of this stage was to gain an overall picture of how sensitive the model is to different configurations and to identify promising regions in the search space.
This broad exploration helps reveal whether stability comes mainly from regularization (dropout/weight decay), optimization dynamics (learning rate, warmup ratio, scheduler), or batch-level training strategies.


### Experiment 1 — Broad Hyperparameter Search

In the first experiment, we defined **wide ranges** for the main hyperparameters in order to capture diverse behaviors of the model.

* **Learning rate (2e-6 → 5e-5):** covers very small steps to moderately aggressive updates.
* **Weight decay (1e-6 → 1e-4):** tests the effect of weak vs. stronger regularization.
* **Batch size (32, 64, 128):** allows comparison between more frequent updates and more stable gradient estimates.
* **Dropout (0.1 → 0.4):** explores different levels of regularization on the classifier head.
* **Number of additional layers (0–1):** checks whether a minimal classification head is sufficient or if one extra layer helps.
* **Gradient clipping (0.5, 1.0, 2.0):** stabilizes training under high variance updates.
* **Class weights (on/off):** evaluates whether weighting the loss improves robustness under class imbalance.

*Motivation:* By searching broadly across these ranges, we aimed to build a strong baseline and observe whether stability and performance are mainly influenced by optimization parameters (learning rate, batch size, gradient clipping) or by regularization (dropout, weight decay, class weights).



**Trail 1**

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 2e-6
learning_rate_to = 5e-5
weight_decay_from = 1e-6
weight_decay_to = 1e-4
patience_choices = [5, 7]
batch_size_choices = [32, 64, 128]
num_layers_choices = [0,1]
dropout_rate_choices =  [0.1, 0.2, 0.3, 0.4]
gradient_clip_val_choices = [0.5, 1.0, 2.0]
use_class_weights_choices = [True, False]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True


In [ ]:
# ============ OBJECTIVE FUNCTION ============

def objective(trial,
              learning_rate_from=learning_rate_from, learning_rate_to=learning_rate_to,
              weight_decay_from=weight_decay_from, weight_decay_to=weight_decay_to,
              patience_choices=patience_choices, batch_size_choices=batch_size_choices,
              num_layers_choices=num_layers_choices, dropout_rate_choices=dropout_rate_choices,
              gradient_clip_val_choices=gradient_clip_val_choices, use_class_weights_choices=use_class_weights_choices,
              enable_gradient_clipping=enable_gradient_clipping, enable_class_weights=enable_class_weights):

    # Suggest hyperparameters using the ranges passed in
    learning_rate = trial.suggest_float("learning_rate", learning_rate_from, learning_rate_to, log=True)
    weight_decay = trial.suggest_float("weight_decay", weight_decay_from, weight_decay_to, log=True)
    patience = trial.suggest_categorical("patience", patience_choices)
    batch_size = trial.suggest_categorical("batch_size", batch_size_choices)
    num_layers = trial.suggest_categorical("num_layers", num_layers_choices)
    dropout_rate = trial.suggest_categorical("dropout_rate", dropout_rate_choices)

    # Optional parameters
    if enable_gradient_clipping:
        gradient_clip_val = trial.suggest_categorical("gradient_clip_val", gradient_clip_val_choices)
    else:
        gradient_clip_val = None

    if enable_class_weights:
        use_class_weights = trial.suggest_categorical("use_class_weights", use_class_weights_choices)
    else:
        use_class_weights = False

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    # Class weights computation
    class_weights = None
    if use_class_weights:
        print("Computing class weights...")
        train_labels = train_df['label'].values
        class_weights = compute_class_weights(train_labels)

    # Dataset and DataLoader
    train_dataset = CoronaTweetDataset(train_df, tokenizer)
    val_dataset = CoronaTweetDataset(eval_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Use custom model with dropout
    model = ConfigurableBertModel(model_name="bert-base-uncased",
        num_labels=5,
        dropout_rate=dropout_rate
    ).to(device)

    # Freeze/unfreeze layers
    for param in model.encoder.parameters():
        param.requires_grad = False
    for param in model.encoder.encoder.layer[-num_layers:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Use class weights in loss if specified
    if use_class_weights and class_weights is not None:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        print("Using weighted CrossEntropyLoss")
    else:
        criterion = nn.CrossEntropyLoss()
        print("Using standard CrossEntropyLoss")

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Initialize wandb
    wandb.init(
        project="bert_base_uncased_tuning",
        config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "dropout_rate": dropout_rate,
            "gradient_clip_val": gradient_clip_val,
            "use_class_weights": use_class_weights,
            "enable_gradient_clipping": enable_gradient_clipping,
            "enable_class_weights": enable_class_weights,
            "architecture": "bert-base-uncased",
            "dataset": "Corona_NLP"
        },
        name=f"trial_{trial.number}"
    )

    # Train with optional gradient clipping
    best_val_accuracy = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion,
        epochs=15, patience=patience, trial=trial,
        gradient_clip_val=gradient_clip_val
    )

    wandb.finish()
    return best_val_accuracy

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study BERT-base-uncased 1:")
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study1.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = current_dir / "best_BERT_base_uncasedmodel1.pt".pt"
best_model = ConfigurableBertModel(
    model_name="bert-base-uncased",
    num_labels=5,
    dropout_rate=best_trial.params["dropout_rate"]
).to(device)

best_model.load_state_dict(torch.load(best_model_path))

torch.save(best_model.state_dict(), save_path)

print(f"\nBest model saved: {best_model_path}")

[I 2025-08-02 16:26:24,200] A new study created in memory with name: no-name-dca47d79-b6db-4f9a-a7b2-11bb263e8d46


Starting Study BERT-base-uncased 1:


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Negative (1): 0.830
Neutral (2): 1.067
Positive (3): 0.721
Extremely Positive (4): 1.243
Using weighted CrossEntropyLoss


Epoch  1/15: Train Acc: 0.6684, Val Acc: 0.8147, Val F1: 0.8206, Gap: -0.1463
Epoch  2/15: Train Acc: 0.8441, Val Acc: 0.8511, Val F1: 0.8550, Gap: -0.0069
Epoch  3/15: Train Acc: 0.8933, Val Acc: 0.8500, Val F1: 0.8531, Gap: 0.0433
Epoch  4/15: Train Acc: 0.9198, Val Acc: 0.8604, Val F1: 0.8636, Gap: 0.0594
Epoch  5/15: Train Acc: 0.9356, Val Acc: 0.8677, Val F1: 0.8722, Gap: 0.0679
Epoch  6/15: Train Acc: 0.9472, Val Acc: 0.8372, Val F1: 0.8413, Gap: 0.1099
Epoch  7/15: Train Acc: 0.9555, Val Acc: 0.8524, Val F1: 0.8564, Gap: 0.1031
Epoch  8/15: Train Acc: 0.9625, Val Acc: 0.8582, Val F1: 0.8620, Gap: 0.1043
Epoch  9/15: Train Acc: 0.9675, Val Acc: 0.8667, Val F1: 0.8702, Gap: 0.1008
Epoch 10/15: Train Acc: 0.9705, Val Acc: 0.8627, Val F1: 0.8666, Gap: 0.1077
Epoch 11/15: Train Acc: 0.9753, Val Acc: 0.8530, Val F1: 0.8579, Gap: 0.1223
Early stopping at epoch 11


Class_Extremely Negative_F1,▁▃▁▅█▄▆▇▆██
Class_Extremely Negative_Precision,▃▂▁█▆▂▇▄▇▇▆
Class_Extremely Negative_Recall,▄▇█▁▅█▂▇▃▄▅
Class_Extremely Positive_F1,▁▄▃▇█▄▇▇▇██
Class_Extremely Positive_Precision,▁▇█▅▆█▇▇▇▇█
Class_Extremely Positive_Recall,▃▃▁██▂▆▆▆▇▆
Class_Negative_F1,▃▇█▇█▇▅▄▇▄▁
Class_Negative_Precision,▅▇█▇▇▅▄▅▆▄▁
Class_Negative_Recall,▁▃▂▃▃▆▇▄▅▅█
Class_Neutral_F1,▁▇▇▇▇▃▅███▆
Class_Neutral_Precision,▁▄▄▃▆█▇▆▄▅▆


[I 2025-08-03 01:20:14,339] Trial 14 finished with value: 0.8677113702623906 and parameters: {'learning_rate': 2.8416691346991923e-05, 'weight_decay': 1.0422703782931357e-06, 'patience': 5, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.1, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 7 with value: 0.874271137026239.



 Best Results:
Validation Accuracy: 0.8743
Best hyperparameters:
  learning_rate: 3.23021771120285e-05
  weight_decay: 1.867444788842222e-05
  patience: 5
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.1
  gradient_clip_val: 0.5
  use_class_weights: True

Best model saved: best_model_trial_7.pt


### Study Results Summary – Refinement after Broad Search

The broad search revealed a clear trade-off between **peak performance** and **overall stability**. While the **best trial** achieved strong validation metrics (≈87% F1, ≈87% accuracy, ≈97% train accuracy), the **average performance across all trials** was substantially lower (≈76% F1, ≈75% accuracy, ≈83% train accuracy).

This gap indicates that although the search space included highly effective configurations, it also contained many unstable or suboptimal settings. The **overfitting gap** was \~10% in the best run, showing the model could generalize well but still risked overfitting when parameters were not balanced.

| Metric              | Best in Trial 1 | Average Trial 1 |
| ------------------- | ------------------------- | --------------------------------- |
| Validation Accuracy | 0.871                     | 0.755                             |
| Validation F1       | 0.874                     | 0.761                             |
| Validation Loss     | 0.600                     | 0.776                             |
| Train Accuracy      | 0.974                     | 0.834                             |
| Train Loss          | 0.088                     | 0.403                             |
| Overfitting Gap     | 0.103                     | 0.080                             |

**Experiment 2 adjustments**
Guided by these insights, we narrowed the hyperparameter ranges to:

* **Learning rate:** 3e-5 → 5e-5, focusing on the region that consistently yielded stable improvements.
* **Dropout:** 0.15–0.25, eliminating extremes that harmed convergence.
* **Batch size:** fixed at 32, which produced the most reliable updates.
* **Extra layers:** fixed at 0, since deeper heads did not improve performance.
* **Gradient clipping:** fixed at 0.5 for stability.
* **Class weights:** enabled to improve handling of class imbalance.

Motivation: This refinement reduced noise from unpromising regions of the parameter space and concentrated optimization on the stable, high-performing zone, balancing **generalization** with **peak performance**.



**Trail 2**

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 3e-5
learning_rate_to = 5e-5
weight_decay_from = 1e-6
weight_decay_to = 5e-5
patience_choices = [5, 7]
batch_size_choices = [32]
num_layers_choices = [0]
dropout_rate_choices =  [0.15, 0.20, 0.25]
gradient_clip_val_choices = [0.5]
use_class_weights_choices = [True]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True


In [ ]:
# ============ OBJECTIVE FUNCTION ============

def objective(trial,
              learning_rate_from=learning_rate_from, learning_rate_to=learning_rate_to,
              weight_decay_from=weight_decay_from, weight_decay_to=weight_decay_to,
              patience_choices=patience_choices, batch_size_choices=batch_size_choices,
              num_layers_choices=num_layers_choices, dropout_rate_choices=dropout_rate_choices,
              gradient_clip_val_choices=gradient_clip_val_choices, use_class_weights_choices=use_class_weights_choices,
              enable_gradient_clipping=enable_gradient_clipping, enable_class_weights=enable_class_weights):

    # Suggest hyperparameters using the ranges passed in
    learning_rate = trial.suggest_float("learning_rate", learning_rate_from, learning_rate_to, log=True)
    weight_decay = trial.suggest_float("weight_decay", weight_decay_from, weight_decay_to, log=True)
    patience = trial.suggest_categorical("patience", patience_choices)
    batch_size = trial.suggest_categorical("batch_size", batch_size_choices)
    num_layers = trial.suggest_categorical("num_layers", num_layers_choices)
    dropout_rate = trial.suggest_categorical("dropout_rate", dropout_rate_choices)

    # Optional parameters
    if enable_gradient_clipping:
        gradient_clip_val = trial.suggest_categorical("gradient_clip_val", gradient_clip_val_choices)
    else:
        gradient_clip_val = None

    if enable_class_weights:
        use_class_weights = trial.suggest_categorical("use_class_weights", use_class_weights_choices)
    else:
        use_class_weights = False

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    # Class weights computation
    class_weights = None
    if use_class_weights:
        print("Computing class weights...")
        train_labels = train_df['label'].values
        class_weights = compute_class_weights(train_labels)

    # Dataset and DataLoader
    train_dataset = CoronaTweetDataset(train_df, tokenizer)
    val_dataset = CoronaTweetDataset(eval_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Use custom model with dropout
    model = ConfigurableBertModel(model_name="bert-base-uncased",
        num_labels=5,
        dropout_rate=dropout_rate
    ).to(device)

    # Freeze/unfreeze layers
    for param in model.encoder.parameters():
        param.requires_grad = False
    for param in model.encoder.encoder.layer[-num_layers:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Use class weights in loss if specified
    if use_class_weights and class_weights is not None:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        print("Using weighted CrossEntropyLoss")
    else:
        criterion = nn.CrossEntropyLoss()
        print("Using standard CrossEntropyLoss")

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Initialize wandb
    wandb.init(
        project="bert_base_uncased_tuning2",
        config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "dropout_rate": dropout_rate,
            "gradient_clip_val": gradient_clip_val,
            "use_class_weights": use_class_weights,
            "enable_gradient_clipping": enable_gradient_clipping,
            "enable_class_weights": enable_class_weights,
            "architecture": "bert-base-uncased",
            "dataset": "Corona_NLP"
        },
        name=f"trial_{trial.number}"
    )

    # Train with optional gradient clipping
    best_val_accuracy = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion,
        epochs=15, patience=patience, trial=trial,
        gradient_clip_val=gradient_clip_val
    )

    wandb.finish()
    return best_val_accuracy

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study BERT-base-uncased 2:")
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study1.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path = current_dir/"best_BERT_base_uncasedmodel2.pt"
best_model = ConfigurableBertModel(
    model_name="bert-base-uncased",
    num_labels=5,
    dropout_rate=best_trial.params["dropout_rate"]
).to(device)

best_model.load_state_dict(torch.load(best_model_path))

torch.save(best_model.state_dict(), save_path)

print(f"\nBest model saved: {best_model_path}")

[I 2025-08-09 08:52:11,088] A new study created in memory with name: no-name-e38c06d4-433e-4088-bee6-73c0f33cdaee


Starting Study BERT-base-uncased 2:


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Negative (1): 0.830
Neutral (2): 1.067
Positive (3): 0.721
Extremely Positive (4): 1.243
Using weighted CrossEntropyLoss


Epoch  1/15: Train Acc: 0.6836, Val Acc: 0.8268, Val F1: 0.8320, Gap: -0.1432
Epoch  2/15: Train Acc: 0.8447, Val Acc: 0.8341, Val F1: 0.8388, Gap: 0.0107
Epoch  3/15: Train Acc: 0.8783, Val Acc: 0.8160, Val F1: 0.8238, Gap: 0.0623
Epoch  4/15: Train Acc: 0.8969, Val Acc: 0.8404, Val F1: 0.8462, Gap: 0.0565
Epoch  5/15: Train Acc: 0.9079, Val Acc: 0.8512, Val F1: 0.8560, Gap: 0.0567
Epoch  6/15: Train Acc: 0.9199, Val Acc: 0.8760, Val F1: 0.8794, Gap: 0.0439
Epoch  7/15: Train Acc: 0.9224, Val Acc: 0.8769, Val F1: 0.8798, Gap: 0.0454
Epoch  8/15: Train Acc: 0.9270, Val Acc: 0.8420, Val F1: 0.8452, Gap: 0.0850
Epoch  9/15: Train Acc: 0.9298, Val Acc: 0.8813, Val F1: 0.8842, Gap: 0.0485
Epoch 10/15: Train Acc: 0.9362, Val Acc: 0.8618, Val F1: 0.8648, Gap: 0.0744
Epoch 11/15: Train Acc: 0.9361, Val Acc: 0.8420, Val F1: 0.8475, Gap: 0.0941
Epoch 12/15: Train Acc: 0.9386, Val Acc: 0.8760, Val F1: 0.8793, Gap: 0.0627
Epoch 13/15: Train Acc: 0.9424, Val Acc: 0.8716, Val F1: 0.8749, Gap: 0.070

Class_Extremely Negative_F1,▁▄▄▇▅█▆▂█▅▇██▆▇
Class_Extremely Negative_Precision,▂▇▃▆▃▅▄▁█▃▆▅▅▇▅
Class_Extremely Negative_Recall,▅▁▆▅█▆▇█▂█▄▆▅▂▆
Class_Extremely Positive_F1,▁▃▃▆▅▇▆▂█▅▆▇▇▇▅
Class_Extremely Positive_Precision,▁▄█▆█▇▆▃▅▅▇▄▆▅▆
Class_Extremely Positive_Recall,▃▄▁▅▃▆▆▃█▅▄▇▆█▄
Class_Negative_F1,▃▁▅▄▅▆█▅▇▅▃▆▄▇▂
Class_Negative_Precision,▅▁█▄▇▆▇█▇▄▃▅▄▅▃
Class_Negative_Recall,▁▅▁▅▂▆▇▁▆█▅▆▇█▃
Class_Neutral_F1,▄▄▁▃▅▇█▆█▇▄█▇█▇
Class_Neutral_Precision,▃▃▁▅▃▅▆▇▄█▄▆▆▆▅


[I 2025-08-09 19:22:30,883] Trial 14 finished with value: 0.8813168124392614 and parameters: {'learning_rate': 4.5235959136129626e-05, 'weight_decay': 4.3632529540259236e-05, 'patience': 7, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.25, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 0 with value: 0.8824101068999028.



 Best Results:
Validation Accuracy: 0.8824
Best hyperparameters:
  learning_rate: 3.745836892235315e-05
  weight_decay: 3.067811223818858e-05
  patience: 7
  batch_size: 32
  num_layers: 0
  dropout_rate: 0.25
  gradient_clip_val: 0.5
  use_class_weights: True

Best model saved: best_model_trial_0.pt



### Study Results Summary – Narrower Search (Experiment 2)

The narrower search confirmed the insights from Experiment 2 and further improved stability.
The **best trial** reached **≈88% validation F1** and **≈88% validation accuracy**, with a reduced **overfitting gap (\~7.7%)**. On average, the trials in this stage performed much closer to the best configuration (**≈86.5% F1, ≈86% accuracy**), which indicates that the refined search space eliminated most of the unstable or poor-performing parameter combinations.

| Metric              | Best Trial Trai 2 | Average Trial 2 |
| ------------------- | ---------------------------- | ------------------------------------ |
| Validation Accuracy | 0.880                        | 0.862                                |
| Validation F1       | 0.884                        | 0.865                                |
| Validation Loss     | 0.497                        | 0.623                                |
| Train Accuracy      | 0.958                        | 0.967                                |
| Train Loss          | 0.126                        | 0.106                                |
| Overfitting Gap     | 0.077                        | 0.105                                |

This stage demonstrated that concentrating around a slightly narrower **learning rate (3.3e-5 → 4.1e-5)**, moderate **dropout (0.15–0.25)**, and fixed **batch size (32)** consistently yielded both high performance and improved generalization.

Motivation: By refining the ranges further, we achieved not only strong peak performance but also reduced variance between trials, which is crucial for reproducibility and reliability in deployment.

**Trail 3**

In [ ]:
# Define your hyperparameter ranges here - easy to change between studies
learning_rate_from = 3.3e-5
learning_rate_to = 4.1e-5
weight_decay_from = 6e-6
weight_decay_to = 4.4e-5
patience_choices = [5, 7]
batch_size_choices = [32]
num_layers_choices = [0]
dropout_rate_choices =  [0.15, 0.25]
gradient_clip_val_choices = [0.5]
use_class_weights_choices = [True]

# Optional features flags
enable_gradient_clipping = True
enable_class_weights = True


In [ ]:
# ============ OBJECTIVE FUNCTION ============

def objective(trial,
              learning_rate_from=learning_rate_from, learning_rate_to=learning_rate_to,
              weight_decay_from=weight_decay_from, weight_decay_to=weight_decay_to,
              patience_choices=patience_choices, batch_size_choices=batch_size_choices,
              num_layers_choices=num_layers_choices, dropout_rate_choices=dropout_rate_choices,
              gradient_clip_val_choices=gradient_clip_val_choices, use_class_weights_choices=use_class_weights_choices,
              enable_gradient_clipping=enable_gradient_clipping, enable_class_weights=enable_class_weights):

    # Suggest hyperparameters using the ranges passed in
    learning_rate = trial.suggest_float("learning_rate", learning_rate_from, learning_rate_to, log=True)
    weight_decay = trial.suggest_float("weight_decay", weight_decay_from, weight_decay_to, log=True)
    patience = trial.suggest_categorical("patience", patience_choices)
    batch_size = trial.suggest_categorical("batch_size", batch_size_choices)
    num_layers = trial.suggest_categorical("num_layers", num_layers_choices)
    dropout_rate = trial.suggest_categorical("dropout_rate", dropout_rate_choices)

    # Optional parameters
    if enable_gradient_clipping:
        gradient_clip_val = trial.suggest_categorical("gradient_clip_val", gradient_clip_val_choices)
    else:
        gradient_clip_val = None

    if enable_class_weights:
        use_class_weights = trial.suggest_categorical("use_class_weights", use_class_weights_choices)
    else:
        use_class_weights = False

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

    # Class weights computation
    class_weights = None
    if use_class_weights:
        print("Computing class weights...")
        train_labels = train_df['label'].values
        class_weights = compute_class_weights(train_labels)

    # Dataset and DataLoader
    train_dataset = CoronaTweetDataset(train_df, tokenizer)
    val_dataset = CoronaTweetDataset(eval_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Use custom model with dropout
    model = ConfigurableBertModel(model_name="bert-base-uncased",
        num_labels=5,
        dropout_rate=dropout_rate
    ).to(device)

    # Freeze/unfreeze layers
    for param in model.encoder.parameters():
        param.requires_grad = False
    for param in model.encoder.encoder.layer[-num_layers:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    # Use class weights in loss if specified
    if use_class_weights and class_weights is not None:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
        print("Using weighted CrossEntropyLoss")
    else:
        criterion = nn.CrossEntropyLoss()
        print("Using standard CrossEntropyLoss")

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Initialize wandb
    wandb.init(
        project="bert_base_uncased_tuning3",
        config={
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "patience": patience,
            "batch_size": batch_size,
            "num_layers": num_layers,
            "dropout_rate": dropout_rate,
            "gradient_clip_val": gradient_clip_val,
            "use_class_weights": use_class_weights,
            "enable_gradient_clipping": enable_gradient_clipping,
            "enable_class_weights": enable_class_weights,
            "architecture": "bert-base-uncased",
            "dataset": "Corona_NLP"
        },
        name=f"trial_{trial.number}"
    )

    # Train with optional gradient clipping
    best_val_accuracy = train_model_with_hyperparams(
        model, train_loader, val_loader, optimizer, criterion,
        epochs=15, patience=patience, trial=trial,
        gradient_clip_val=gradient_clip_val
    )

    wandb.finish()
    return best_val_accuracy

In [ ]:
# ============ RUN THE STUDY ============
print("Starting Study BERT-base-uncased 3:")
study1 = optuna.create_study(direction="maximize")
study1.optimize(objective, n_trials=15)


# ============ RUN THE STUDY ============
best_trial = study1.best_trial
best_model_path = f"best_model_trial_{best_trial.number}.pt"
print(f"\n Best Results:")
print(f"Validation Accuracy: {best_trial.value:.4f}")
print(f"Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")
save_path =current_dir / "best_BERT_base_uncasedmodel3.pt"
best_model = ConfigurableBertModel(
    model_name="bert-base-uncased",
    num_labels=5,
    dropout_rate=best_trial.params["dropout_rate"]
).to(device)

best_model.load_state_dict(torch.load(best_model_path))

torch.save(best_model.state_dict(), save_path)

print(f"\nBest model saved: {best_model_path}")

[I 2025-08-09 21:41:15,445] A new study created in memory with name: no-name-be081988-75c7-4850-8218-41cc7ed39c40


Starting Study BERT-base-uncased 3:


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Epoch  1/15: Train Acc: 0.6818, Val Acc: 0.7754, Val F1: 0.7840, Gap: -0.0936
Epoch  2/15: Train Acc: 0.8419, Val Acc: 0.7756, Val F1: 0.7869, Gap: 0.0663
Epoch  3/15: Train Acc: 0.8843, Val Acc: 0.8671, Val F1: 0.8704, Gap: 0.0172
Epoch  4/15: Train Acc: 0.9049, Val Acc: 0.8596, Val F1: 0.8642, Gap: 0.0453
Epoch  5/15: Train Acc: 0.9178, Val Acc: 0.8602, Val F1: 0.8637, Gap: 0.0576
Epoch  6/15: Train Acc: 0.9273, Val Acc: 0.8630, Val F1: 0.8666, Gap: 0.0643
Epoch  7/15: Train Acc: 0.9328, Val Acc: 0.8701, Val F1: 0.8733, Gap: 0.0626
Epoch  8/15: Train Acc: 0.9357, Val Acc: 0.8635, Val F1: 0.8664, Gap: 0.0722
Epoch  9/15: Train Acc: 0.9397, Val Acc: 0.8633, Val F1: 0.8673, Gap: 0.0763
Epoch 10/15: Train Acc: 0.9427, Val Acc: 0.8648, Val F1: 0.8691, Gap: 0.0780
Epoch 11/15: Train Acc: 0.9443, Val Acc: 0.8681, Val F1: 0.8713, Gap: 0.0762
Epoch 12/15: Train Acc: 0.9462, Val Acc: 0.8635, Val F1: 0.8681, Gap: 0.0827
Epoch 13/15: Train Acc: 0.9482, Val Acc: 0.8686, Val F1: 0.8722, Gap: 0.079

Class_Extremely Negative_F1,▁▂▆▅▄▄▆▄▆█▆█▆
Class_Extremely Negative_Precision,▂▁▄▆▂▃█▃▇▆▄█▄
Class_Extremely Negative_Recall,▅█▆▂█▇▁▇▂▅▇▃▇
Class_Extremely Positive_F1,▁▂▆▆▄▆▇▅▇█▆█▆
Class_Extremely Positive_Precision,▁█▃▁▂▆▁▂▄▅▂▆▂
Class_Extremely Positive_Recall,▂▁▆▇▅▅█▆▇▇▆▇▇
Class_Negative_F1,▁▇▆▆██▃▆▅▇▆▅▇
Class_Negative_Precision,▂▇▄█▆▆▁▅▅▄▆▄▅
Class_Negative_Recall,▃▃▇▁▅▄█▄▄▇▃▅▆
Class_Neutral_F1,▁▁█▇█▇██▇▇█▇█
Class_Neutral_Precision,▃▁▆▅▇▆██▅█▆▆▇


[I 2025-08-10 05:33:30,467] Trial 10 finished with value: 0.8701409135082604 and parameters: {'learning_rate': 3.845053433215533e-05, 'weight_decay': 3.951665800508248e-05, 'patience': 5, 'batch_size': 32, 'num_layers': 0, 'dropout_rate': 0.25, 'gradient_clip_val': 0.5, 'use_class_weights': True}. Best is trial 0 with value: 0.8813168124392614.


Computing class weights...
Computing class weights for 32925 samples
Label distribution: [4385 7934 6170 9137 5299]
Class weights computed:
Extremely Negative (0): 1.502
Negative (1): 0.830
Neutral (2): 1.067
Positive (3): 0.721
Extremely Positive (4): 1.243
Using weighted CrossEntropyLoss


Epoch  1/15: Train Acc: 0.6796, Val Acc: 0.8245, Val F1: 0.8294, Gap: -0.1448
Epoch  2/15: Train Acc: 0.8453, Val Acc: 0.8477, Val F1: 0.8523, Gap: -0.0023



### Results Comparison – Experiment 1 → 3

| Metric              | Exp. 1 (Broad) | Exp. 2 (Narrow) | Exp. 3 (Narrower) |
| ------------------- | -------------- | --------------- | ----------------- |
| Validation Accuracy | 0.871          | 0.871           | **0.880**         |
| Validation F1       | 0.874          | 0.874           | **0.884**         |
| Validation Loss     | 0.600          | 0.600           | **0.497**         |
| Train Accuracy      | 0.974          | 0.974           | 0.958             |
| Train Loss          | 0.088          | 0.088           | 0.126             |
| Overfitting Gap     | 0.103          | 0.103           | **0.077**         |

---

**Overall Summary — Experiment 1 to 3**
The progression from Experiment 1 (broad search) to Experiment 2 (narrowed search) already stabilized the model, eliminating many unstable configurations. Experiment 3 further refined the ranges (learning rate, dropout, weight decay), but the results show **no substantial improvement compared to Experiment 2**.

* Validation F1 and Accuracy increased only marginally (≈+1%).
* Validation Loss improved slightly (0.600 → 0.497).
* Overfitting Gap decreased from \~10% to \~7.7%, suggesting slightly better generalization.
* However, Train Accuracy dropped slightly, indicating the model was already near its optimal point.

**Conclusion:**
Although Experiment 3 showed slight gains in validation metrics, these improvements were marginal and came at the cost of a small drop in training accuracy. The results suggest that the model had already reached a performance plateau by Experiment 2, and additional narrowing of the hyperparameter ranges in Experiment 3 did not yield substantial benefits. Therefore, we decided to stop the hyperparameter search at this stage, as further tuning would likely bring only diminishing returns without meaningful improvements in generalization or stability.
